In [41]:
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot
import torch.nn as nn
from numpy import fft

#Preprocessing real and complex values
'''
__call__ turns this obejct into a function
__init__ is the equivalent of a constructor in Java
'''
class FFTTransform:
    def __call__(self,x):
        x = x.squeeze(0)
        x_fft = torch.fft.fft2(x)
        return x_fft

base_transforms = transforms.Compose([
    transforms.ToTensor()
])

complex_transforms = transforms.Compose([
    transforms.ToTensor(),
    FFTTransform()
])

batch_size = 32

#Real Values
train_dataset = datasets.MNIST(root='./data',download=True, train=True, transform=base_transforms)
test_dataset  = datasets.MNIST(root='./data',download=True, train=False, transform=base_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
#Complex Values
train_dataset_i = datasets.MNIST(root='./data',download=True, train=True, transform =complex_transforms)
test_dataset_i = datasets.MNIST(root='./data',download=True, train=False, transform=complex_transforms)

train_loader_i = DataLoader(train_dataset_i, batch_size=batch_size, shuffle=True)
test_loader_i = DataLoader(test_dataset_i, batch_size=batch_size, shuffle=True)


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 14.8MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 547kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.87MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.87MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
import torch.nn as nn
import torch

batch_size = 32

#Real valued data
class RealMLP(nn.Module):
    def __init__(self, bias=True):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 128),  
            nn.ReLU(),
            nn.Linear(128, 10),   
        )
#This final output is a tensor of logits
    def forward(self, x):
        return self.net(x)

'''
Imaginary Valued Data

Consists of two parts:
1. Building a custom linear dense layer that can accommodate
imaginary values
2. Building model architecture akin to RealMLP, but with custom dense layer 
instead of nn.Linear(x,y)

There are two additional areas to explore: 
1. Experiment with handling the bias function differently
2. Experiment with different complex activation functions 
'''


class ComplexLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.real_weight = nn.Linear(in_features, out_features)
        self.complex_weight = nn.Linear(in_features, out_features)

#Ensure x is a torch.complex 
    def forward(self, x):
        x_real = x.real
        x_imag = x.imag

        real_out = self.real_weight(x_real) - self.complex_weight(x_imag)
        imag_out = self.real_weight(x_imag) + self.complex_weight(x_real)

        return torch.complex(real_out, imag_out)


class ComplexMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = ComplexLinear(784, 128)
        self.fc2 = ComplexLinear(128, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x.real) + 1j * torch.relu(x.imag)  
        x = self.fc2(x)
        return x


        




In [86]:
model = RealMLP()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    total_loss = 0
    correct = 0
    total = 0

    for x_batch, y_batch in train_loader:
        x_batch = x_batch.reshape(batch_size, x_batch.shape[2]*x_batch.shape[3])
        preds = model(x_batch)
        
        loss = criterion(preds, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (preds.argmax(dim=1) == y_batch).sum().item()
        total += y_batch.size(0)

    acc = correct / total * 100
    print(f"Epoch {epoch+1}: Loss={total_loss:.4f}, Accuracy={acc:.2f}%")


def evaluate(model, test_loader):
    model.eval()  
    correct = 0
    total = 0

    with torch.no_grad():  
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.view(x_batch.size(0), -1)
            outputs = model(x_batch)
            preds = outputs.argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    acc = correct / total * 100
    return acc

val_acc = evaluate(model, test_loader)
print(f"Epoch {epoch+1}, Train Loss: {loss:.4f}, Val Accuracy: {val_acc:.2f}%")


Epoch 1: Loss=558.9601, Accuracy=91.73%
Epoch 2: Loss=242.9819, Accuracy=96.18%
Epoch 3: Loss=167.1954, Accuracy=97.33%
Epoch 4: Loss=122.9449, Accuracy=98.02%
Epoch 5: Loss=97.7867, Accuracy=98.41%
Epoch 6: Loss=75.0241, Accuracy=98.78%
Epoch 7: Loss=62.5222, Accuracy=98.94%
Epoch 8: Loss=49.4900, Accuracy=99.21%
Epoch 9: Loss=39.8574, Accuracy=99.32%
Epoch 10: Loss=33.1670, Accuracy=99.45%
Epoch 10, Train Loss: 0.0318, Val Accuracy: 97.90%


In [ ]:
#Complex Model 1
'''Experimenting with bias:

CVNN
If bias = True, accuracy ~ 97.02
If bias = False, accuracy ~97.00

If bias = True, accuracy ~ 97.90
If bias = False, accuracy ~97.83

Negligble across models/dataset
'''
complex_model = ComplexMLP()
complex_criterion = nn.CrossEntropyLoss()
complex_optimizer = torch.optim.Adam(complex_model.parameters(), lr=1e-3)

for epoch in range(10):
    total_loss = 0
    correct = 0
    total = 0

    for x_batch, y_batch in train_loader_i:
        x_batch = x_batch.reshape(batch_size, x_batch.shape[1]*x_batch.shape[2])
        preds = complex_model(x_batch)
        preds = preds.real
                
        complex_loss = complex_criterion(preds, y_batch)

        complex_optimizer.zero_grad()
        complex_loss.backward()
        complex_optimizer.step()

        total_loss += complex_loss.item()
        correct += (preds.argmax(dim=1) == y_batch).sum().item()
        total += y_batch.size(0)

    acc = correct / total * 100
    print(f"Epoch {epoch+1}: Loss={total_loss:.4f}, Accuracy={acc:.2f}%")


def imaginary_evaluate(model, test_loader):
    model.eval()  
    correct = 0
    total = 0

    with torch.no_grad():  
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.view(x_batch.size(0), -1)
            outputs = model(x_batch)
            outputs = outputs.real
            preds = outputs.argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    acc = correct / total * 100
    return acc

val_acc = imaginary_evaluate(complex_model, test_loader_i)
print(f"Epoch {epoch+1}, Train Loss: {total_loss:.4f}, Val Accuracy: {val_acc:.2f}%")



Epoch 1: Loss=440.9363, Accuracy=93.23%
Epoch 2: Loss=196.9401, Accuracy=96.88%
Epoch 3: Loss=153.0192, Accuracy=97.69%
Epoch 4: Loss=130.6195, Accuracy=98.03%
Epoch 5: Loss=106.2810, Accuracy=98.47%
Epoch 6: Loss=113.8958, Accuracy=98.56%
Epoch 7: Loss=103.0648, Accuracy=98.71%
Epoch 8: Loss=90.0500, Accuracy=98.95%
Epoch 9: Loss=82.7339, Accuracy=99.03%
Epoch 10: Loss=86.4420, Accuracy=99.08%
Epoch 10, Train Loss: 86.4420, Val Accuracy: 96.43%
